# **Sexism in "Survivor"**

Let's talk about my favorite television show, Survivor. It's been around for 20 years, it has 40 seasons, and yes, it is still going on (Season 41 is being filmed as I type this!). For those who do not know, Survivor is a reality television show where they put between 16-20 strangers (well, usually strangers) on an island (which is nowadays always a beach, but they used to go to deserts, rainforests, etc.) and they have to live with each other in their own little society. The fun part about Survivor is, they get to vote people off to not live with them anymore! They have challenges and start off split into tribes, and the losing tribe has to go to "Tribal Council", the place where they vote someone off. Then, they merge into 1 tribe and keep voting people off until there are only 2 or 3 people left. The people who have been voted off make up the jury and they vote for someone in the final 2/3 to win the season. And that's a qick gist of Survivor! 


Now even though it is my favorite show, it does come with many problems. One of the main ones is sexism and the inequality that exists between men and women on the show. This exists in both the game of Survivor, in terms of how fair it is for men versus women, as well as in the show of Survivor, in how men versus women are portrayed on television. Also, a monumental distinction to make is that the game of Survivor is what happens out there on the island and how the production behind the scenes controls the game, through different twists and such. While the show of Survivor is the product that we, as the viewer, see on television. The show is a product of what happens in the game (for that season) and is the result of cutting hundreds of hours of footage to display exactly what they, behind the show, want to show to the viewer. This distinction between the game and the show of Survivor is extremely important to clarify. 


Also, a note on what I said about cutting hundreds of hours of footage and why that matters. Each season lasts 39 days and there are 13 or 14 episodes every season. So, on average, one 42 minute episode contains footage from 3 days worth of gameplay, or 72 hours. And that is just counting 1 camera ... they have many, many cameras. So, the producers and editors have a boatlad of footage to use however they want to display their reality television characters. This means, not everything shown has to be entirely truthful or presented to us in the correct timeline and order that it happened. For example, they could get  clip of Jessica saying how annoying Phillip is on Day 20 and edit that to put it in Day 10 and try to build up a (fake) storyline that Jessica wants to get Phillip out on Day 10. And this is the reality of reality television. 

## The questions I will be answering: 

- How and where does sexism exist in the game of Survivor?
- How and where does sexism exist in the show of Survivor?
- Who is at "fault" and how can these inequalities be alleviated? (MODIFY)

## Why is this important? 

Survivor can often be viewed as a microcosm of society. As in, the disparity between men and women here in Survivor represents a bigger issue about our society as whole. Most of the imbalances that exist in the show and game of Survivor are because of their existence in society as a whole too, beyond just Survivor. So, when I dive into certain issues like how women are portrayed on the television show of Survivor, that is relevant because it is connected to our pre-conceived beliefs of societal gender roles and has a much bigger impact than just on the game of Survivor. The data and information I will present in this project reflect some of the inherent bias in how we perceive men and women, at least in the more quantifiable ways. This will still be much more of a data-driven project, rather than a philosophical paper on gender roles, but it is important to draw the connection between sexism in Survivor and how that translates to sexism in real life.

## Dataset I will use

I actually have my own Google Sheets spreadsheet I have been sporadically working on the past few years (because I really just am that big of a nerd of Survivor). I will use a specific few tabs on that sheet to highlight data involving a gender disparity in different aspects of Survivor. The [spreadsheet](https://docs.google.com/spreadsheets/d/1Gb377DdGNfzct0Nt9BYMF-moLXcxFxhOnePBQhIJ73s/edit#gid=741044694) is fairly messy right now, as not everything is up to date, but the pink tabs are the ones I will use for this project. I could make a new spreadsheet document just for the tabs I will use for that project if that makes it an easier resource to present, or maybe there is another way that would be better. I'm open to suggestions!

A considerable amount of the data in that spreadsheet does just come from my own memory of having watched every season of Survivior and somehow remembering loads of minute details. But, the [Survivor Wiki](https://survivor.fandom.com/wiki/Main_Page) and [The True Dork Times](http://www.truedorktimes.com/survivor/boxscores/index.htm) were also resources I used to collect this data. The Survivor Wiki has data on every single season specifically, while the True Dork Times link has loads of different categories for neat Survivor statistics.

https://drive.google.com/drive/u/1/folders/0B8Xzl82K1TP8fmItS2RoYWUxeW1YSmZoUXVQSldNMTJnUEVSV1Zvd2xYaFpLYnViOWJ1RXM

## Ethical concerns about the problem, question, and data

Certain meninists could be offended by how pro-women this study may seem (although it really is intended to be more of a pro-equality study) and could try to excuse the disparity between men and women on Survivor. Certain people who wouldn't want to acknowledge sexism or flaws in Survivor would not be very receptive to listening to what this data presents. This isn't meant to say women are better than men (or vice versa) or that women need to win a certain amount of seasons or have a certain amount of airtime. But it is rather an acknowledgement of the flaws in the show and hopefully a step towards gender equality on Survivor, and in society in general.

Another issue is who exactly is at fault here and how do we remedy the sexism in Survivor? Is this on producers to include less advantages that historically benefit men? Is it on the editors to give women more screen time and not let the men dominate the airtime? Is it the fault of women for not being better Survivor players and not being dynamic enough characters to get more airtime? I doubt it, but I think our inherent bias plays a role there and is a considerably large factor as to why men are generally perceived as more entertaining, and get more screen time. 

And this data project could be seen as a way to knock on men and put an asterick next to their achievements in Survivor, which is not the intention. Generally, they have it easier in some ways, but loads of men have done very well in Survivor because of their own doing. This is more so meant to acknowledge the inherent bias that the editors, players, and even us as the viewers have when it comes to gender roles in Survivor. 

One more concern is, why does this actually matter? And it matters because the sexism we see in Survivor (both in the game and the show) goes far beyond just the show and it represents the dynamic of sexual inequality in society. 